# Klasifikacija SMS poruka

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import nltk
import string

In [3]:
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import feature_extraction

U prethodnom primeru smo videli na koje načine možemo da kreiramo `vokabular`, skup svih reči koje se pojavljuju u kolekciji tekstova kojom raspolažemo. Obično su ove reči leksikografski uređene, nalik popisu reči u rečnicima. U ovom primeru želimo da vidimo kako možemo doći do reprezentacija pogodnih za primenu algoritama mašinskog učenja. 

Tekstualna kolekcija nad kojom ćemo dalje eksperimentisati sadrži dve rečenice. To mogu biti i dva pasusa, dva dokumenta ili bilo koje druge tekstualne forme. 

In [4]:
corpus = ['This is so so so interesting!', 'Interesting is always better than non interesting.']

## Vreća reci (engl. Bag of Words)

Pretpostavimo da u vokabularu imamo `|V|` reči i da su one leksikografski uređene. Rečima možemo dodeliti redne brojeve koji odgovaraju pozicijama na kojima se nalaze. Rečenicu teksta stoga možemo predstaviti kao vektor dimenzije `|V|` u kojem se na poziciji `i` nalazi `1` ukoliko je `i`-ta reč vokabulara prisutna u rečenici ili `0` ukoliko se `i`-ta reč vokabulara ne nalazi u njoj. Ovakva reprezentacija rečenica (u opštem slučaju bilo kojih fragmenata teksta) gubi informaciju o povezanosti reči (rečenice *Tom likes Mary* i *Mary likes Tom* imaju iste reprezentacije i sasvim različitu semantiku) kao i informaciju o broju pojavljivanja određenih reči (što može biti važno za određivanje npr. teme teksta). Zato se ova reprezentacija može nadgraditi reprezentacijom u kojoj se na poziciji `i` nalazi `broj pojavljivanja reči` ili `0` ukoliko se ta reč ne javlja u rečenici. Ovakve reprezentacije i dalje nisu savršene jer ne uzimaju u obzir dužinu teksta - ukoliko je tekst dug reči se mogu pojavljivati neopravdano sa većom frekvencijom, i obratno.

<img src='assets/count_vectorizer.png'>

Bibiloteka `scikit-learn` sadrži klasu `CountVectorizer` koja nudi funkcionalnost vektorizacije teksta u model vreće reči sa frekvencijama (brojem pojavljivanja).

## TF-IDF (Term Frequency - Inverse Document Frequency)

Reprezentacija rečenica vrećom reči (sa frekvencijama) se može unaprediti prelaskom na reprezentaciju sa trežinama. Težine bi trebale da reflektuju važnost reči u samoj rečenici, ali i važnost u odnosu na druge rečenice kolekcije. Jedna takva vektorska reprezentacija je Tf-Idf reprezentacija koja se uspešno koristi u mnogim NLP zadacima, na primer, u pretraživanju informacija (engl. information retrieval) ili istraživanju teksta (engl. text mining).
Tf je skraćenica od `term frequency`, a Idf skraćenica od `inverse document frequency`. U primeru koji mi razmatramo dokumenti su zapravo rečenice, ali se time ne gubi na opštosti ovog pristupa.

Neka je dalje: 
- $D = \{d_1, d_2\}$ - kolekcija rečenica/dokumenata
- $d_1 = $ 'This is so so so interesting!' - prva rečenica/dokument
- $d_2 = $ 'Interesting is always better than non interesting.' - druga rečenica/dokument



### Izračunavanje  Tf-Idf težina
Težina *tf-idf* je proizvod dva faktora:
- $Tf(t, d)$ - koji predstavlja količnik broja pojavljivanja reči/termina $t$ u dokumentu $d$ i ukupnog broja reči u dokumentu $d$
$$Tf(t, d) = \frac{count(t)}{|d|}$$
- $Idf(t, D)$ - koji predstavlja logaritam količnika ukupnog broja dokumenata skupa $D$ i broja dokumenata (iz skupa 
$D$) u kojima se pojavljuje termin $t$
$$Idf(t, D) = log(\frac{|D|}{N_t})$$


Dakle, $$TfIdf(t, d, D) =Tf(t, d)\cdot Idf(t, D)$$

Važnost termina (reči) raste sa rastom broja njegovog pojavljivanja u dokumentu, ali se otežava njegovim ukupnim brojem pojavljivanja u celokupnom skupu dokumenata (rečima koje se pojavljuju u većem broju dokumenata pridružuje se manja vrednost jer je i njihova diskriminativnost manja).

Postoje i mnoge varijacije na ovu temu o kojima se može više pročitati u članku na [Vikipediji](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). 

#### Primer
Razmotrimo dokument $d$ koji sadrži 100 reči u kojima se reč *mačka* pojavljuje 3 puta.

Tf("mačka", $d$) = $\frac{3}{100}$ = 0.03

Pretpostavimo da imamo *10 miliona* dokumenata i da se reč "mačka" pojavljuje u 1000 njih. 

Idf("mačka", $D$) = $\log{(\frac{10 000 000}{1 000})} = 4$

Mera *tf-idf* predstavlja proizvod nadjenih vrednosti, odnosno $TfIdf = 0.03 \cdot 4 = 0.12$.

Bibiloteka `scikit-learn` sadrži klasu `TfIdfVectorizer` koja nudi funkcionalnost vektorizacije teksta na gore opisani način.

## Kasifikacija SMS poruka na poželjne i nepoželjne

U ovom primeru želimo da testiramo efektnost reprezentacija u zadatku klasifikacije sms poruka na spam i ne-spam poruke. 

### Učitavanje i priprema skupa podataka

U direktorijumu `smsspam` nalazi se datoteka `SMSSpamCollection` u kojoj se nalaze pomenute porukice.

Struktura je jedne linije dokumenta je sledeća:
```txt
LABELA\tSADRZAJ\n
```
Odnosno, prvo je navedena labela 
- `spam` - ukoliko je poruka spam
- `ham` - ukoliko poruka nije spam

potom karakter `\t`, a potom i sadržaj sms poruke sve do karaktera `\n`.


### Model logističke regresije

### Linearni SVM model

### Model k-najbližih suseda

Funkcija kojom se prikazuju reči vokabulara najzaslužnije za klasifikovanje poruka na spam i ne-spam poruke.

In [5]:
def visualize_coefficients(title, classifier, feature_names, n_top_features=25):
    coefs = classifier.coef_.ravel()
    
    positive_coefficients_indices = np.argsort(coefs)[-n_top_features:]
    negative_coefficients_indices = np.argsort(coefs)[:n_top_features]
    
    interesting_coefficients_indices =\
        np.hstack([negative_coefficients_indices,
                   positive_coefficients_indices])
    
    plt.figure(figsize=(15,5))
    plt.title(title)
    colors = ['orange' if c < 0 else 'cadetblue' for c in coefs[interesting_coefficients_indices]]
    plt.bar(np.arange(2 * n_top_features),
            coefs[interesting_coefficients_indices], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(2*n_top_features), feature_names[interesting_coefficients_indices], rotation=60, ha='right')